# Stacked LSTMs for Time Series Classification with TensorFlow

### Loading Libraries

In [13]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
import pandas_datareader.data as web

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Warning
import warnings

# Path
from pathlib import Path

# Scikit-Learn
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

# TensorFlow
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Dense, LSTM, Input, concatenate, Embedding, Reshape, BatchNormalization

In [14]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')

if gpu_devices:
    print('Using GPU')
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)
else:
    print('Using CPU')

Using GPU


In [15]:
np.random.seed(42)

idx = pd.IndexSlice

sns.set_style('whitegrid')

In [19]:
results_path = Path('results', 'lstm_embeddings')

if not results_path.exists():
    results_path.mkdir(parents=True)

### Data

In [23]:
data = pd.read_hdf('data.h5', 'returns_weekly')

In [25]:
data['ticker'] = pd.factorize(data.index.get_level_values('ticker'))[0]

In [27]:
data['month'] = data.index.get_level_values('date').month

data = pd.get_dummies(data, columns=['month'], prefix='month')

In [30]:
data.info()

### Train-Test Split

In [45]:
window_size=52

sequence = list(range(1, window_size+1))

ticker = 1

months = 12

n_tickers = data.ticker.nunique()

In [47]:
train_data = data.drop('fwd_returns', axis=1).loc[idx[:, :'2016'], :]

test_data = data.drop('fwd_returns', axis=1).loc[idx[:, '2017'],:]

In [49]:
X_train = [
    train_data.loc[:, sequence].values.reshape(-1, window_size , 1),
    train_data.ticker,
    train_data.filter(like='month')
]

y_train = train_data.label

[x.shape for x in X_train], y_train.shape

In [51]:
X_test = [
    test_data.loc[:, list(range(1, window_size+1))].values.reshape(-1, window_size , 1),
    test_data.ticker,
    test_data.filter(like='month')
]

y_test = test_data.label
[x.shape for x in X_test], y_test.shape

#### Defining Model Architecture

In [53]:
K.clear_session()

In [55]:
n_features = 1

In [57]:
returns = Input(shape=(window_size, n_features),
                name='Returns')

tickers = Input(shape=(1,),
                name='Tickers')

months = Input(shape=(12,),
               name='Months')

### LSTM Layers

In [61]:
lstm1_units = 25

lstm2_units = 10

In [73]:
lstm1 = LSTM(units=lstm1_units, 
             input_shape=(window_size, 
                          n_features), 
             name='LSTM1', 
             dropout=.2,
             return_sequences=True)(returns)

lstm_model = LSTM(units=lstm2_units, 
             dropout=.2,
             name='LSTM2')(lstm1)

#### Embedding Layer

In [76]:
ticker_embedding = Embedding(input_dim=n_tickers, 
                             output_dim=5, 
                             input_length=1)(tickers)

ticker_embedding = Reshape(target_shape=(5,))(ticker_embedding)

### Concatenating Model Components

In [79]:
merged = concatenate([lstm_model, 
                      ticker_embedding, 
                      months], name='Merged')

bn = BatchNormalization()(merged)
hidden_dense = Dense(10, name='FC1')(bn)

output = Dense(1, name='Output', activation='sigmoid')(hidden_dense)

rnn = Model(inputs=[returns, tickers, months], outputs=output)

In [81]:
rnn.summary()

### Training Model

In [85]:
optimizer = tf.keras.optimizers.RMSprop(lr=0.001,
                                        rho=0.9,
                                        epsilon=1e-08,
                                        decay=0.0)

In [87]:
rnn.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy', 
                     tf.keras.metrics.AUC(name='AUC')])

In [89]:
lstm_path = (results_path / 'lstm.classification.h5').as_posix()

checkpointer = ModelCheckpoint(filepath=lstm_path,
                               verbose=1,
                               monitor='val_AUC',
                               mode='max',
                               save_best_only=True)

In [97]:
early_stopping = EarlyStopping(monitor='val_AUC', 
                              patience=5,
                              restore_best_weights=True,
                              mode='max')

In [99]:
training = rnn.fit(X_train,
                   y_train,
                   epochs=50,
                   batch_size=32,
                   validation_data=(X_test, y_test),
                   callbacks=[early_stopping, checkpointer],
                   verbose=1)

In [101]:
loss_history = pd.DataFrame(training.history)

In [105]:
def which_metric(m):
    return m.split('_')[-1]

In [107]:
fig, axes = plt.subplots(ncols=3, figsize=(18,4))

for i, (metric, hist) in enumerate(loss_history.groupby(which_metric, axis=1)):
    hist.plot(ax=axes[i], title=metric)
    axes[i].legend(['Training', 'Validation'])

sns.despine()
fig.tight_layout()
fig.savefig(results_path / 'lstm_stacked_classification', dpi=300);
plt.grid()
plt.show()

### Evaluating Model Performance

In [112]:
test_predict = pd.Series(rnn.predict(X_test).squeeze(), index=y_test.index)

In [114]:
roc_auc_score(y_score=test_predict, y_true=y_test)

In [116]:
((test_predict>.5) == y_test).astype(int).mean()

In [118]:
spearmanr(test_predict, y_test)[0]